# Preparation

In [1]:
import time
prep_start_time = time.time()

In [2]:
!mkdir pretrained
!mkdir dataset
!mkdir logs

In [3]:
!rm -r pretrained/mobilenet
!mkdir pretrained/mobilenet
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz -O pretrained/mobilenet_v2_1.0_224.tgz
!tar -xvf pretrained/mobilenet_v2_1.0_224.tgz -C pretrained/mobilenet/
!rm pretrained/mobilenet_v2_1.0_224.tgz

rm: cannot remove `pretrained/mobilenet': 그런 파일이나 디렉터리가 없습니다
--2018-07-08 04:21:36--  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.25.208, 2404:6800:4004:81b::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.25.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78306834 (75M) [application/x-tar]
Saving to: ‘pretrained/mobilenet_v2_1.0_224.tgz’

pretrained/mobilene 100%[===================>]  74.68M  40.9MB/s    in 1.8s    

2018-07-08 04:21:38 (40.9 MB/s) - ‘pretrained/mobilenet_v2_1.0_224.tgz’ saved [78306834/78306834]

./
./mobilenet_v2_1.0_224_eval.pbtxt
./mobilenet_v2_1.0_224.ckpt.data-00000-of-00001
./mobilenet_v2_1.0_224_frozen.pb
./mobilenet_v2_1.0_224.tflite
./mobilenet_v2_1.0_224_info.txt
./mobilenet_v2_1.0_224.ckpt.meta
./mobilenet_v2_1.0_224.ckpt.index


In [1]:
!rm -r pretrained/resnet
!mkdir pretrained/resnet
!wget http://download.tensorflow.org/models/resnet_v2_101_2017_04_14.tar.gz -O pretrained/resnet_v2_101_2017_04_14.tar.gz
!tar -xvf pretrained/resnet_v2_101_2017_04_14.tar.gz -C pretrained/resnet/
!rm pretrained/resnet_v2_101_2017_04_14.tar.gz

rm: pretrained/resnet: No such file or directory
--2018-07-10 15:10:59--  http://download.tensorflow.org/models/resnet_v2_101_2017_04_14.tar.gz
Resolving download.tensorflow.org... 172.217.161.48
Connecting to download.tensorflow.org|172.217.161.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499948384 (477M) [application/x-tar]
Saving to: ‘pretrained/resnet_v2_101_2017_04_14.tar.gz’

pretrained/resnet_v 100%[===================>] 476.79M  10.8MB/s    in 49s     

2018-07-10 15:11:49 (9.78 MB/s) - ‘pretrained/resnet_v2_101_2017_04_14.tar.gz’ saved [499948384/499948384]

x resnet_v2_101.ckpt
x train.graph
x eval.graph


In [7]:
!rm -r dataset/surreal
!mkdir -p dataset/surreal/cmu
!aws s3 cp s3://syd-surreal/cmu dataset/surreal/cmu --recursive --only-show-errors

In [0]:
print('Prep took: ', time.time() - prep_start_time, 'ms')

In [0]:
!ls -alh

# 새 섹션

In [1]:
%load_ext autoreload
%autoreload

import functools, operator, copy
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
from personlab import surreal0 as surreal
from personlab import config
from personlab.nets import resnet_v2
from personlab.nets.mobilenet import mobilenet_v2
from personlab.tensor_info import INPUT_SINGLE_TENSOR_INFO as INPUT_TENSOR_INFO
from personlab.tensor_info import OUTPUT_SINGLE_TENSOR_INFO as OUTPUT_TENSOR_INFO
from personlab.display import *
tf.reset_default_graph()

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
surreal_feeder = surreal.SurrealFeeder()
surreal_feeder.start()
types = tuple(t['type'] for t in INPUT_TENSOR_INFO)
input_tensors = tf.data.Dataset.from_generator(surreal_feeder.single_retrive, types) \
                    .batch(config.BATCH_SIZE) \
                    .prefetch(config.PREFETCH_SIZE) \
                    .make_one_shot_iterator() \
                    .get_next()
#                    .apply(tf.contrib.data.shuffle_and_repeat(config.SHUFFLE_SIZE, config.REPEAT_COUNT)) \
tensors = {}
for tensor, info in zip(input_tensors, INPUT_TENSOR_INFO):
    tensor.set_shape(info['shape'])
    tensors[info['name']] = tensor
print(tensors)

worker thread start!
{'image': <tf.Tensor 'IteratorGetNext:0' shape=(10, 401, 401, 3) dtype=float32>, 'hm': <tf.Tensor 'IteratorGetNext:1' shape=(10, 401, 401, 14) dtype=float32>, 'so_x': <tf.Tensor 'IteratorGetNext:2' shape=(10, 401, 401, 14) dtype=float32>, 'so_y': <tf.Tensor 'IteratorGetNext:3' shape=(10, 401, 401, 14) dtype=float32>, 'mo_x': <tf.Tensor 'IteratorGetNext:4' shape=(10, 401, 401, 28) dtype=float32>, 'mo_y': <tf.Tensor 'IteratorGetNext:5' shape=(10, 401, 401, 28) dtype=float32>}


In [3]:
config.STRIDE = 8

MD_H = int((config.TAR_H-1)//config.STRIDE)+1
MD_W = int((config.TAR_W-1)//config.STRIDE)+1

DEPTH = [ti['shape'][-1] for ti in OUTPUT_TENSOR_INFO]
RESULT_SHAPE = (config.BATCH_SIZE, MD_H, MD_W, sum(DEPTH))
RESULT_SIZE = functools.reduce(operator.mul, RESULT_SHAPE[1:])
OUTPUT_SHAPE = (config.BATCH_SIZE, config.TAR_H, config.TAR_W, sum(DEPTH))
OUTPUT_SIZE = functools.reduce(operator.mul, OUTPUT_SHAPE[1:])


In [4]:
def bilinear(indices):
    oy = tf.clip_by_value(indices[1], 0, MD_H-1-1e-8)
    ox = tf.clip_by_value(indices[2], 0, MD_W-1-1e-8)
    iy = [tf.floor(oy), tf.ceil(oy + 1e-9)]
    ix = [tf.floor(ox), tf.ceil(ox + 1e-9)]
    idx_p = []
    for y in iy:
        for x in ix:
            indices[1] = y
            indices[2] = x
            idx = tf.cast(tf.stack(indices, axis=-1), tf.int32)
            p = (1 - tf.abs(y - oy)) * (1 - tf.abs(x - ox))
            idx_p.append((idx, p))
    return idx_p

def gather_bilinear(params, indices):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.gather_nd(params, idx)
        res.append(r * p)
    return tf.add_n(res)

def scatter_bilinear(params, indices, shape):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.scatter_nd(idx, params, shape)
        if len(r.shape) > len(p.shape):
            p = tf.expand_dims(p, axis=-1)
        res.append(r * p)
    return tf.add_n(res)

In [5]:
def resize(tensor):
    return tf.image.resize_images(
        tensor,
        (config.TAR_H, config.TAR_W),
        method=tf.image.ResizeMethod.BILINEAR,
        align_corners=True)

with slim.arg_scope(mobilenet_v2.training_scope(is_training=True)):
    model_output, _ = mobilenet_v2.mobilenet_base(tensors['image'], output_stride=config.STRIDE)
    print('model_output', model_output)
    hm_pred = slim.conv2d(model_output, surreal.NUM_KP, [1, 1], activation_fn=tf.sigmoid)
    so_x_pred = slim.conv2d(model_output, surreal.NUM_KP, [1, 1], activation_fn=None)
    so_y_pred = slim.conv2d(model_output, surreal.NUM_KP, [1, 1], activation_fn=None)
    mo_x_pred = slim.conv2d(model_output, surreal.NUM_EDGE, [1, 1], activation_fn=None)
    mo_y_pred = slim.conv2d(model_output, surreal.NUM_EDGE, [1, 1], activation_fn=None)

    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)
    b, y, x, _ = np.mgrid[:config.BATCH_SIZE, :MD_H, :MD_W, :surreal.NUM_EDGE]
    print(surreal.EDGES[:, 0])
    i = np.tile(surreal.EDGES[:, 0], [config.BATCH_SIZE, MD_H, MD_W, 1])
    for _ in range(config.NUM_RECURRENT):
        mo_p = [b, y+mo_y_pred, x+mo_x_pred, i]
        mo_x_pred = gather_bilinear(so_x_pred, mo_p) + mo_x_pred
        mo_y_pred = gather_bilinear(so_y_pred, mo_p) + mo_y_pred
    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)
    hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred = [resize(x) for x in [hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred]]
    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)
    so_x_pred, so_y_pred, mo_x_pred, mo_y_pred = [x * config.STRIDE for x in [so_x_pred, so_y_pred, mo_x_pred, mo_y_pred]]
    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)

model_output Tensor("MobilenetV2/Conv_1/Relu6:0", shape=(10, 51, 51, 1280), dtype=float32)
Tensor("Conv/Sigmoid:0", shape=(10, 51, 51, 14), dtype=float32) Tensor("Conv_1/BatchNorm/FusedBatchNorm:0", shape=(10, 51, 51, 14), dtype=float32) Tensor("Conv_2/BatchNorm/FusedBatchNorm:0", shape=(10, 51, 51, 14), dtype=float32) Tensor("Conv_3/BatchNorm/FusedBatchNorm:0", shape=(10, 51, 51, 28), dtype=float32) Tensor("Conv_4/BatchNorm/FusedBatchNorm:0", shape=(10, 51, 51, 28), dtype=float32)
[ 0  1  1  2  3  2  3  4  5  8  8  9 10 11  1  2  3  4  5  8  9  6  7  9
 10 11 12 13]
Tensor("Conv/Sigmoid:0", shape=(10, 51, 51, 14), dtype=float32) Tensor("Conv_1/BatchNorm/FusedBatchNorm:0", shape=(10, 51, 51, 14), dtype=float32) Tensor("Conv_2/BatchNorm/FusedBatchNorm:0", shape=(10, 51, 51, 14), dtype=float32) Tensor("add_12:0", shape=(10, 51, 51, 28), dtype=float32) Tensor("add_15:0", shape=(10, 51, 51, 28), dtype=float32)
Tensor("resize_images/ResizeBilinear:0", shape=(10, 401, 401, 14), dtype=float32

In [6]:
hm_loss = - tf.reduce_mean(tensors['hm'] * tf.log(hm_pred + 1e-9) + (1 - tensors['hm']) * tf.log(1 - hm_pred + 1e-9))
so_loss = tf.abs(tensors['so_x'] - so_x_pred) / config.RADIUS + tf.abs(tensors['so_y'] - so_y_pred) / config.RADIUS
mo_loss = tf.abs(tensors['mo_x'] - mo_x_pred) / config.RADIUS + tf.abs(tensors['mo_y'] - mo_y_pred) / config.RADIUS

disc_only = tf.cast(tensors['hm'], tf.float32)
disc_size = tf.reduce_sum(disc_only, axis=[1, 2]) + 1e-9
so_loss = tf.reduce_mean(tf.reduce_sum(so_loss * disc_only, axis=[1, 2]) / disc_size)

disc_only = tf.cast(tf.gather(tensors['hm'], surreal.EDGES[:, 0], axis=-1), tf.float32)
disc_size = tf.reduce_sum(disc_only, axis=[1, 2]) + 1e-9
mo_loss = tf.reduce_mean(tf.reduce_sum(mo_loss * disc_only, axis=[1, 2]) / disc_size)

In [7]:
total_loss = hm_loss * 4.0 + so_loss * 1.0 + mo_loss * 0.5

In [8]:
b_i, kp_i = 0, 0
args = [tensors['image'][b_i], hm_pred[b_i], tensors['hm'][b_i],
        mo_x_pred[b_i], mo_y_pred[b_i]]
image_tensor1 = tf.py_func(summary_image, args, tf.float32)
image_tensor1.set_shape([None, None, 4])
tf.summary.image('sum', tf.expand_dims(image_tensor1, 0))

<tf.Tensor 'sum:0' shape=() dtype=string>

In [9]:
tf.summary.scalar('losses/hm_loss', hm_loss)
tf.summary.scalar('losses/so_loss', so_loss)
tf.summary.scalar('losses/mo_loss', mo_loss)
tf.summary.scalar('losses/total_loss', total_loss)

<tf.Tensor 'losses/total_loss:0' shape=() dtype=string>

In [10]:
optimizer = tf.train.AdamOptimizer()
train_op = slim.learning.create_train_op(total_loss, optimizer)

checkpoint_path = 'pretrained/mobilenet/mobilenet_v2_1.0_224.ckpt'
variables = slim.get_model_variables()
restore_map = {}
for v in variables:
    if not v.name.startswith('MobilenetV2'):
        continue
    org_name = v.name.split(':')[0]
    restore_map[org_name] = v
    print(org_name, ':', v.name)
init_assign_op, init_feed_dict = slim.assign_from_checkpoint(checkpoint_path, restore_map)

MobilenetV2/Conv/weights : MobilenetV2/Conv/weights:0
MobilenetV2/Conv/BatchNorm/gamma : MobilenetV2/Conv/BatchNorm/gamma:0
MobilenetV2/Conv/BatchNorm/beta : MobilenetV2/Conv/BatchNorm/beta:0
MobilenetV2/Conv/BatchNorm/moving_mean : MobilenetV2/Conv/BatchNorm/moving_mean:0
MobilenetV2/Conv/BatchNorm/moving_variance : MobilenetV2/Conv/BatchNorm/moving_variance:0
MobilenetV2/expanded_conv/depthwise/depthwise_weights : MobilenetV2/expanded_conv/depthwise/depthwise_weights:0
MobilenetV2/expanded_conv/depthwise/BatchNorm/gamma : MobilenetV2/expanded_conv/depthwise/BatchNorm/gamma:0
MobilenetV2/expanded_conv/depthwise/BatchNorm/beta : MobilenetV2/expanded_conv/depthwise/BatchNorm/beta:0
MobilenetV2/expanded_conv/depthwise/BatchNorm/moving_mean : MobilenetV2/expanded_conv/depthwise/BatchNorm/moving_mean:0
MobilenetV2/expanded_conv/depthwise/BatchNorm/moving_variance : MobilenetV2/expanded_conv/depthwise/BatchNorm/moving_variance:0
MobilenetV2/expanded_conv/project/weights : MobilenetV2/expand

In [11]:
!rm -r logs/mb_single_log_180708_045856

In [12]:
import time, os, shutil
log_dir="logs/mb_single_log_180708_045856"
#log_dir = 'logs/mb_single_log_' + time.strftime("%y%m%d_%H%M%S", time.gmtime())
os.mkdir(log_dir)

In [ ]:
def InitAssignFn(sess):
    sess.run(init_assign_op, init_feed_dict)
    
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=True)):
    tf.contrib.slim.learning.train(train_op,
                                   log_dir,
                                   init_fn=InitAssignFn,
                                   log_every_n_steps=100,
                                   save_summaries_secs=60,
                                  )

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path logs/mb_single_log_180708_045856/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 0.455181
INFO:tensorflow:Recording summary at step 27.
INFO:tensorflow:global_step/sec: 0.550016
INFO:tensorflow:Recording summary at step 60.
INFO:tensorflow:global_step/sec: 0.546039
INFO:tensorflow:Recording summary at step 93.
INFO:tensorflow:global step 99: loss = 3.5820 (1.826 sec/step)
INFO:tensorflow:global_step/sec: 0.54762
INFO:tensorflow:Recording summary at step 126.
INFO:tensorflow:global_step/sec: 0.532872
INFO:tensorflow:Recording summary at step 158.
INFO:tensorflow:global_step/sec: 0.549876
INFO:tensorflow:Recording summary at step 191.
INFO:tensorfl

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/personlab/surreal0.py", line 183, in _process
    res = self.transformer.run(frames, kp_list, kp_idx, kp_seg)
  File "/home/ec2-user/personlab/transformer.py", line 73, in run
    mo_y = np.zeros([num_frame, config.TAR_H, config.TAR_W, surreal.NUM_EDGE]).astype(np.int16)
MemoryError



INFO:tensorflow:global_step/sec: 0.547821
INFO:tensorflow:Recording summary at step 290.
INFO:tensorflow:global step 299: loss = 3.0869 (1.827 sec/step)
INFO:tensorflow:Saving checkpoint to path logs/mb_single_log_180708_045856/model.ckpt
INFO:tensorflow:Recording summary at step 322.
INFO:tensorflow:Recording summary at step 355.


In [ ]:
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################

In [1]:

saver = tf.train.Saver()
checkpoint_path = '/Users/minsubsim/work/personlab/archive/mb_single_log_180707_005358/model.ckpt-28629/model.ckpt-28629'
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    m_out, img, hm, so_x, so_y, mo_x, mo_y = sess.run([model_output, tensors['image'], hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred])


NameError: name 'tf' is not defined